In [1]:
import datasets
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [2]:
dataset = load_dataset("hackathon-pln-es/es_tweets_laboral")

Found cached dataset parquet (/home/andres/.cache/huggingface/datasets/hackathon-pln-es___parquet/hackathon-pln-es--es_tweets_laboral-a483c2bb98fb482d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Select N examples per class (8 in this case)
train_ds = dataset["train"].shuffle(seed=42).select(range(8 * 2))
test_ds = dataset["test"]

Loading cached shuffled indices for dataset at /home/andres/.cache/huggingface/datasets/hackathon-pln-es___parquet/hackathon-pln-es--es_tweets_laboral-a483c2bb98fb482d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c577e030effce602.arrow


In [4]:
def rename_dataset(dataset):
    return datasets.Dataset.from_pandas(
        dataset.to_pandas()
        .drop(columns = 'entities')
        .rename(columns = {'intent':'label'})
    )

In [5]:
train_ds = rename_dataset(train_ds)
test_ds = rename_dataset(test_ds)

In [6]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    load_in_8bit=True
)

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=8,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [7]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

TypeError: '<' not supported between instances of 'NoneType' and 'str'

In [ ]:
metrics

In [ ]:
train_ds.num_rows

In [ ]:
test_ds.num_rows